In [21]:
import numpy as np
import scipy
import scipy.linalg   # SciPy Linear Algebra Library

# band matrix
A = np.array([ [7, 3, 0, 0], [3, 8, 1, 0], [0, 1, 4, -1], [0, 0, -1, 6] ])
u=1;l=1;m=4
b = np.array([1,2,3,4]).reshape(4,1)
ab = np.zeros((u+l+1,m))

# band matrix를 풀기 위한(scipy.linalg.solve_banded를 사용하기 위한) 변형 행렬 ab
for i in range(m):
    for j in range(m):
        if (u+i-j)>=0 and (u+i-j)<(u+l+1):
            ab[u+i-j,j]=A[i,j]
print(b)
print(ab)
P, L, U = scipy.linalg.lu(A)

print(A)
print(P)
print(L)
print(U)

[[1]
 [2]
 [3]
 [4]]
[[ 0.  3.  1. -1.]
 [ 7.  8.  4.  6.]
 [ 3.  1. -1.  0.]]
[[ 7  3  0  0]
 [ 3  8  1  0]
 [ 0  1  4 -1]
 [ 0  0 -1  6]]
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
[[ 1.          0.          0.          0.        ]
 [ 0.42857143  1.          0.          0.        ]
 [ 0.          0.14893617  1.          0.        ]
 [ 0.          0.         -0.25966851  1.        ]]
[[ 7.          3.          0.          0.        ]
 [ 0.          6.71428571  1.          0.        ]
 [ 0.          0.          3.85106383 -1.        ]
 [ 0.          0.          0.          5.74033149]]


In [23]:
import numpy as np
# LU분해가 소수점이 들어간 작은 수일 경우 pivoting을 안해주면 오차가 크게 나는 것으로 보임
# ax=b 에서 np.linalg.solve는 a must be square and of full-rank 여야함.
# 그래서 np.linalg.lstsq()를 사용함.
# lstsq() 명령은 행렬 A와 b를 모두 인수로 받고 뒤에서 설명할 최소자승문제(least square problem)의 답 x, 잔차제곱합(residual sum of squares) resid, 랭크(rank) rank, 특잇값(singular value) s를 반환.
# lstsq() 명령을 사용하는 것이 inv() 명령을 사용하는 것보다 수치오차가 적고 코드도 간단하므로 선형 연립방정식의 해를 구할 때도 lstsq() 명령을 사용하는 것을 권장
# scipy.linalg.solve_banded는 triagonal band system을 풀 수 있음
# LAPACK(Linear Algebra PACKage) LAPACK기반

%time mat_solve = np.linalg.solve(A,b)
%time mat_lstsq = np.linalg.lstsq(A,b)[0]
%time mat_triband = scipy.linalg.solve_banded((1,1),ab,b)
print(mat_solve)
print(mat_lstsq)
print(mat_triband)
# 그런데 위의 3가지 방법 모두 같은 값이 나오네...ㅜ

Wall time: 0 ns
Wall time: 1 ms
Wall time: 0 ns
[[0.10202117]
 [0.09528393]
 [0.93166506]
 [0.82194418]]
[[0.10202117]
 [0.09528393]
 [0.93166506]
 [0.82194418]]
[[0.10202117]
 [0.09528393]
 [0.93166506]
 [0.82194418]]


In [4]:
import numpy as np
import math

# LU 분해 구현
def Solve_SE(matA, matb):
    rowA = len(matA)
    colA = len(matA.T)
    rowB = len(matb)
    if (rowA != colA) | (rowB != colA):
        raise NotImplementedError
    else:
        pass
    L = np.zeros((rowA, colA))
    U = np.zeros((rowA, colA))
 
    for j in range(0, len(L.T)):
        for i in range(0, len(L)):
            if j==0:
                L[i, j] = matA[i, j]
                U[j, i] = matA[j, i] / L[0, 0]
            else:
                if i < j:
                    L[i, j] = 0
                else:
                    Lsum = 0
                    for k in range(0, j):
                        Lsum = Lsum + L[i, k]*U[k, j]
                    L[i, j] = matA[i, j] - Lsum
            if i < j:
                U[j, i] = 0
            elif i == j:
                U[j, i] = 1
            else:
                Usum = 0
                for k in range(0, i):
                    Usum = Usum + L[k, i]*U[j, k]
                U[j, i] = (matA[j, i] - Usum) /L[j, j]
 
 
    modB = np.zeros((3, len(matb.T)))
    modB[0] = matb[0] / L[0, 0]
 
    for i in range(0, len(modB)):
        sumofB = 0
        for k in range(0, i):
            sumofB = sumofB + L[i, k]*modB[k]
        modB[i] = (matb[i] - sumofB) / L[i, i]
 
    x = np.zeros((3, len(modB.T)))
    for no in range(len(x)-1, 0-1, -1):
        sum=0
        for i in range(0, len(x)):
            sum = sum + U[no, i] * x[i]
        x[no] = modB[no] - sum
    return x
 
k=50; r=0.10; t=5/12; sigma=0.4; smax=100; q=0
m=20; n=10
ds = smax/m
dt = t/n
 
f = np.zeros((n+1, m+1))
 
for j in range(1, m):
    f[n, j] = max(k - j * ds, 0)
 
for i in range(0, n+1):
    f[i, 0] = k
for i in range(0, n+1):
    f[i, m] = 0

j=3
a3 = 0.5 * dt * ((r-q) * j - (sigma ** 2) * (j ** 2))
b3 = 1 + ((sigma * j) ** 2) * dt + r * dt
c3 = 0.5 * dt * (-(r-q) * j - (sigma ** 2) * (j ** 2))
 
j=2
a2 = 0.5 * dt * ((r-q) * j - (sigma ** 2) * (j ** 2))
b2 = 1 + ((sigma * j) ** 2) * dt + r * dt
c2 = 0.5 * dt * (-(r-q) * j - (sigma ** 2) * (j ** 2))
 
j=1
a1 = 0.5 * dt * ((r-q) * j - (sigma ** 2) * (j ** 2))
b1 = 1 + ((sigma * j) ** 2) * dt + r * dt
c1 = 0.5 * dt * (-(r-q) * j - (sigma ** 2) * (j ** 2))
 
matA = np.zeros((3, 3))
matA[0, 0] = b3; matA[0, 1] = a3; matA[0, 2] = 0
matA[1, 0] = c2; matA[1, 1] = b2; matA[1, 2] = a2
matA[2, 0] = 0; matA[2, 1] = c1; matA[2, 2] = b1
 
matb = np.zeros((3, 1))
matb[0] = f[4, 3] - c3*f[3, 4]
matb[1] = f[4, 2]
matb[2] = f[4, 1] - a1*f[3, 0]
 
matx = Solve_SE(matA, matb)
 
for i in range(len(matx)):
    print('option price: ', round(float(matx[i]), 4))

option price:  0.0
option price:  0.0006
option price:  0.0618


In [ ]:
# 3 band system 계산해보기